In [124]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from google.cloud import bigquery

## BQ client setup

In [125]:
def estimate_bigquery_query_cost(bq_client, query):
    
    job_config = bigquery.QueryJobConfig()
    job_config.dry_run = True
    job_config.use_query_cache = False
    query_job = bq_client.query(

        (
           query
        ),
        job_config=job_config,
    )
    
    cost_euros = (query_job.total_bytes_processed / 1024 ** 4) * 6

    print(f"{query_job.total_bytes_processed} bytes will be processed , cost ~{cost_euros}$")

In [126]:
bigquery_client= bigquery.Client(project="ingka-chscesda-solar-dev") # ingka-energy-solar-dev # ingka-energy-analytics-dev

### Data mapping

- (`ingka-web-analytics-prod.web_data_v2.hits_events_and_pages`): 
    - Landing on clean-energy (solar) tables, and session data
    - Searched string queries in the internal search bar
- (`ingka-web-analytics-prod.web_data_v2.sessions`) - session data. Contains `icm_id`, `session_id` and `visitor_id`.
- (`ikea-icm-prod.customer_master_golden_record_masked.v_cm_customer`) - Contains data on customer. *** Reach to to Mikael Bergkvist
for info and access permissions *** 


TODO:
- Map google searches (`ingka-do-seo-prod.integrated_search.gsc_est_revenue`) to IKEA user session activity.
- Find transactions data, to see what visitors who reached CES goal have also bought.

## Landings data

In [74]:
# Visited pages
query_string = """
SELECT 
    date_hit
    , visitor_id
    , session_id
    -- , page_urls
FROM (
    SELECT 
    date_hit
    , visitor_id
    , session_id
    , page_urls
    , EXISTS(SELECT * FROM UNNEST(page_urls) AS x WHERE REGEXP_CONTAINS(x, ".+?clean-energy.+")) as exist
    FROM (
        SELECT 
            date_hit
            , visitor_id
            , session_id
            , ARRAY_AGG(page_url) as page_urls
        FROM `ingka-web-analytics-prod.web_data_v2.hits_events_and_pages` 
        WHERE 
            date_hit = '2022-07-01' 
            -- AND website_market_short in ('se')
            AND event_category like '%page%'
            AND page_url is not NULL
        GROUP BY
            date_hit
            , visitor_id
            , session_id
        -- ORDER BY visitor_id
    ) pages
)
WHERE exist is TRUE

"""

estimate_bigquery_query_cost(bigquery_client, query_string) 

35755206733 bytes will be processed , cost ~0.19511502650675538$


In [ ]:
# Results to pandas df
job_config = bigquery.QueryJobConfig(
    use_legacy_sql=False
)

df = (
    bigquery_client.query(query_string, job_config)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=True,
    )
)

df

In [75]:
# Write to table
table_id="ingka-energy-analytics-dev.ces_da_playground.visits_with_landing_temp"

job_config = bigquery.QueryJobConfig(
    allow_large_results=True, destination=table_id, use_legacy_sql=False,
    write_disposition = "WRITE_TRUNCATE"
)


bigquery_client.query(
    query_string, 
    job_config
).result()

## Searches data

In [83]:
# Get SQL string filter condition
product_string_queries = [
    'solar', 'sol', 'sun',
    'sunpower', 'svea', 
    'solstrale', 'SOLSTRÅLE', 
    'solpaneler', 'energy', 'electricity', 
]
string_exp = ' OR '.join([f'"{exp.lower()}" in UNNEST(internal_search_terms)' for exp in product_string_queries])
print(string_exp,'\n')


# Download query results.
query_string = """
SELECT *
FROM (
    SELECT 
        date_hit
        , visitor_id
        , session_id
        , website_market_short
        , website_language_short
        , ARRAY_AGG(DISTINCT LOWER(internal_search_term)) as internal_search_terms
    FROM `ingka-web-analytics-prod.web_data_v2.hits_events_and_pages` 
    WHERE 
        date_hit = '2022-07-02' 
        AND website_market_short in ('us','se')
        AND event_category like '%search%'
        AND internal_search_type = 'hard_search'
        AND internal_search_term is not Null
    GROUP BY
        date_hit
        , visitor_id
        , session_id
        , website_market_short
        , website_language_short
    -- ORDER BY visitor_id
) searches
WHERE 
    {exp}
""".format(exp=string_exp)

estimate_bigquery_query_cost(bigquery_client, query_string) 

"solar" in UNNEST(internal_search_terms) OR "sol" in UNNEST(internal_search_terms) OR "sun" in UNNEST(internal_search_terms) OR "sunpower" in UNNEST(internal_search_terms) OR "svea" in UNNEST(internal_search_terms) OR "solstrale" in UNNEST(internal_search_terms) OR "solstråle" in UNNEST(internal_search_terms) OR "solpaneler" in UNNEST(internal_search_terms) OR "energy" in UNNEST(internal_search_terms) OR "electricity" in UNNEST(internal_search_terms) 

24252107253 bytes will be processed , cost ~0.13234297832059383$


In [107]:
# Write to table
table_id="ingka-energy-analytics-dev.ces_da_playground.searches_temp"

job_config = bigquery.QueryJobConfig(
    allow_large_results=True, destination=table_id, use_legacy_sql=False,
    write_disposition = "WRITE_TRUNCATE"
)


bigquery_client.query(
    query_string, 
    job_config=job_config
).result()

In [84]:
job_config = bigquery.QueryJobConfig(
    use_legacy_sql=False
)

df = (
    bigquery_client.query(query_string, job_config)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=True,
    )
)

df

,date_hit,visitor_id,session_id,website_market_short,website_language_short,internal_search_terms
0,2022-07-02,4650446983238553072,46504469832385530721656785359,us,en,"[solar, full bed frame, closet storage, queen ..."
1,2022-07-02,6028586445314919834,60285864453149198341656718317,us,en,"[planter, sun shade, mirror, solar powered lig..."
2,2022-07-02,7097798603530976994,70977986035309769941656788963,us,en,"[duvet cover, outdoor, carafe, solar]"
3,2022-07-02,7129006291150804045,71290062911508040451656771479,us,en,"[sommar, solar]"
4,2022-07-02,3038983306843116284,30389833068431162841656715193,us,en,"[bar cart, patio, sun, corkscrew, sollerön, sa..."
5,2022-07-02,9127416546358969478,91274165463589694781656734853,us,en,"[new, solar, pink bedspread, summer, pink , in..."
6,2022-07-02,8539080557331644690,85390805573316446901656773917,us,en,"[solar, äpplarö]"
7,2022-07-02,6553147298137891716,65531472981378917161656761174,se,sv,[solstråle]
8,2022-07-02,5394257884940827038,53942578849408270381656715678,us,en,[solar]
9,2022-07-02,1201535132985493757,12015351329854937571656785157,us,en,[solar]


### Search keywords conducted in sessions that included a landing in _any_ clean-energy page

In [116]:
# Visited pages
query_string = """
SELECT 
    pages.date_hit
    , pages.visitor_id
    , pages.session_id
    -- , pages.page_urls
    , pages.page_titles
    -- , searches.website_market_short
    -- , searches.website_language_short
    , searches.internal_search_terms
FROM (
    SELECT 
    date_hit
    , visitor_id
    , session_id
    , page_urls
    , page_titles
    , EXISTS(SELECT * FROM UNNEST(page_urls) AS x WHERE REGEXP_CONTAINS(x, ".+?clean-energy.+")) as exist -- ".+?solar-systems.+"
    FROM (
        SELECT 
            date_hit
            , visitor_id
            , session_id
            , ARRAY_AGG(page_url) as page_urls
            , ARRAY_AGG(page_title) as page_titles
        FROM `ingka-web-analytics-prod.web_data_v2.hits_events_and_pages` 
        WHERE 
            date_hit >= "{date_hit}"
            -- AND website_market_short in ('se')
            AND event_category like '%page%'
            AND page_url is not NULL
        GROUP BY
            date_hit
            , visitor_id
            , session_id
        ) 
    ) pages
INNER JOIN (
    SELECT *
    FROM (
        SELECT 
            date_hit
            , visitor_id
            , session_id
            , website_market_short
            , website_language_short
            , ARRAY_AGG(DISTINCT LOWER(internal_search_term)) as internal_search_terms
        FROM `ingka-web-analytics-prod.web_data_v2.hits_events_and_pages` 
        WHERE 
            date_hit >= "{date_hit}"
            AND website_market_short in ('se')
            AND event_category like '%search%'
            AND internal_search_type = 'hard_search'
            AND internal_search_term is not Null
        GROUP BY
            date_hit
            , visitor_id
            , session_id
            , website_market_short
            , website_language_short
    ) 
    -- WHERE 
    --     {exp}
) searches
ON searches.visitor_id = pages.visitor_id AND searches.session_id = pages.session_id
WHERE 
    pages.date_hit >= "{date_hit}"
    AND pages.exist is TRUE
""".format(date_hit='2022-07-13', exp=string_exp)
estimate_bigquery_query_cost(bigquery_client, query_string) 

51792858761 bytes will be processed , cost ~0.28263198379681853$


In [117]:
# Results to pandas df
job_config = bigquery.QueryJobConfig(
    use_legacy_sql=False
)

df = (
    bigquery_client.query(query_string, job_config)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=True,
    )
)

df

,date_hit,visitor_id,session_id,page_titles,internal_search_terms
0,2022-07-13,6041708102981422944,60417081029814229441657704288,[Solceller – paket och priser för din villa oc...,[solseller]
1,2022-07-13,7642600808297616739,76426008082976167391657687188,"[SKÅDIS Klämma, vit - IKEA, SKÅDIS Klämma, vit...","[kabelsamlare, obegränsad, kabellåda]"


In [122]:
import nltk
from nltk.corpus import stopwords


In [123]:
nltk.download('stopwords')
stopwords.words('swedish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/or.levkovich/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['och',
 'det',
 'att',
 'i',
 'en',
 'jag',
 'hon',
 'som',
 'han',
 'på',
 'den',
 'med',
 'var',
 'sig',
 'för',
 'så',
 'till',
 'är',
 'men',
 'ett',
 'om',
 'hade',
 'de',
 'av',
 'icke',
 'mig',
 'du',
 'henne',
 'då',
 'sin',
 'nu',
 'har',
 'inte',
 'hans',
 'honom',
 'skulle',
 'hennes',
 'där',
 'min',
 'man',
 'ej',
 'vid',
 'kunde',
 'något',
 'från',
 'ut',
 'när',
 'efter',
 'upp',
 'vi',
 'dem',
 'vara',
 'vad',
 'över',
 'än',
 'dig',
 'kan',
 'sina',
 'här',
 'ha',
 'mot',
 'alla',
 'under',
 'någon',
 'eller',
 'allt',
 'mycket',
 'sedan',
 'ju',
 'denna',
 'själv',
 'detta',
 'åt',
 'utan',
 'varit',
 'hur',
 'ingen',
 'mitt',
 'ni',
 'bli',
 'blev',
 'oss',
 'din',
 'dessa',
 'några',
 'deras',
 'blir',
 'mina',
 'samma',
 'vilken',
 'er',
 'sådan',
 'vår',
 'blivit',
 'dess',
 'inom',
 'mellan',
 'sådant',
 'varför',
 'varje',
 'vilka',
 'ditt',
 'vem',
 'vilket',
 'sitta',
 'sådana',
 'vart',
 'dina',
 'vars',
 'vårt',
 'våra',
 'ert',
 'era',
 'vilkas']

1. Break page titles into keywords (or n-grams?)
2. Count keywords co-occurences on session level, put pairs into matrices.
3. Calculate cos-similarity between pairs.
4. Cluster (kmeans, dbscan, hdbscan)
